# Setup
**This notebook serves as a demo and tutorial for the yielengine package.**
It illustrates the following methods:
- EDA
- Boruta feature selection
- Shap clustering feature selection
- simulaiton

In [ ]:
# Print the used verion of Python to be sure that you are using the yield-engine environement
import sys
print(sys.executable)

In [ ]:
PATH_YIELD_ENGINE = 'src'

def set_paths() -> None:
    """
    set correct working directory and python path when started from within PyCharm
    """
    import sys
    import os
    
    if 'cwd' not in globals():
        # noinspection PyGlobalUndefined
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir)
        os.chdir(cwd)
    
    print(f"working dir is '{os.getcwd()}'")
                             
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    
    print(f"added `{sys.path[0]}` to python paths")

set_paths()

In [ ]:
import logging
import os    
import re
import pprint
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error
from typing import *

from yieldengine import Sample
from yieldengine.dendrogram import DendrogramDrawer
from yieldengine.dendrogram.style import FeatMapStyle, LineStyle
from yieldengine.df.pipeline import PipelineDF
from yieldengine.preprocessing.encode import OneHotEncoderDF
from yieldengine.preprocessing.impute import SimpleImputerDF, MissingIndicatorDF
from yieldengine.preprocessing.selection import BorutaDF
from yieldengine.preprocessing.compose import ColumnTransformerDF
from yieldengine.model.inspection import ModelInspector
from yieldengine.model.prediction import ModelFitCV
from yieldengine.model.selection import Model, ModelGrid, ModelRanker, summary_report, ModelEvaluation, ModelScoring
from yieldengine.model.validation import CircularCrossValidator
from yieldengine.visualization.eda import plot_ecdf, plot_ecdf_df, plot_hist_df
from yieldengine.preprocessing.outlier import OutlierRemoverDF
from yieldengine.preprocessing import FunctionTransformerDF
from yieldengine.simulation import UnivariateSimulation

In [ ]:
# Enlarge the width of the cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Only show matplolib warning logging messages, otherwise there are too many
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
log = logging.getLogger(__name__)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 8]

In [ ]:
IQR_THRESHOLD = 1.5

# Load raw data, EDA and data preprocessing

## Raw data

In [ ]:
DATA_DIR = os.path.join('data', 'ames-housing-dataset')
RAW_DATA_CSV = os.path.join(DATA_DIR, 'train.csv')
TARGET = "SalePrice"

print(f"Read {RAW_DATA_CSV}")
raw_df = pd.read_csv(RAW_DATA_CSV)

## EDA

In [ ]:
print(f"Shape of the raw data: {raw_df.shape}")
raw_df.head()

In [ ]:
raw_df.columns

## Description of the variables
The variable *description_dict* is a dictionary whose keys are the column names of the dataset and the values are description of the columns

In [ ]:
DESCRIPTION = os.path.join(DATA_DIR, 'data_description.txt')
with open(DESCRIPTION, 'r') as f:
    description = f.read()

# Create a dictionary containing description of the variables
splitted = description.split()
idx_begin = [i for i, w in enumerate(description.split()) if w.endswith(':')]
description_dict = {
       splitted[idx_begin[i]][:-1]: ' '.join(splitted[idx_begin[i]+1: idx_begin[i+1]]) for i in range(len(idx_begin)-1)
}

pprint.pprint(description_dict)

## Sort by sold date
We sort the samples by selling date

In [ ]:
df_tmp = raw_df.copy()
df_tmp['YrSold'].value_counts().sort_index().plot(kind='bar')

In [ ]:
df_tmp['DateSold'] = pd.to_datetime(df_tmp['YrSold']*100 + df_tmp['MoSold'], format='%Y%m')
df_tmp.groupby('DateSold')[TARGET].mean().plot();

## Data Prep

In [ ]:
dataset_df = raw_df.copy()
# Sort by the date
dataset_df = dataset_df.sort_values(by=['YrSold', 'MoSold'])
# Drop the Id column and date columns
dataset_df = dataset_df.drop(['Id', 'YrSold', 'MoSold'], axis=1)

In [ ]:
# remove infinity values
dataset_df = dataset_df.replace([np.inf, -np.inf], np.nan)

Overview of the types of the features

In [ ]:
features_df = dataset_df.drop(columns=TARGET)
cat_features = features_df.select_dtypes(['category', object]).columns
num_features = features_df.select_dtypes('number').columns
print (f"{features_df.shape[1] - len(cat_features) - len(num_features)} not a number or category")
del features_df

In [ ]:
for col_name in cat_features:
    dataset_df.loc[:, col_name] = (
        dataset_df.loc[:, col_name].astype(object).fillna('missing value').astype(str).str.strip().str.lower().astype('category')
    )
# output datasets
dt = dataset_df.dtypes.rename('dtype').astype(str)
dt.groupby(by=dt.values).count().to_frame()

For simplicity we select only numerical features

In [ ]:
dataset_df = dataset_df[list(num_features) + [TARGET]]

## Remove outliers

In [ ]:
outlier_remover = OutlierRemoverDF(iqr_multiple=IQR_THRESHOLD)

In [ ]:
# Filter out the rows of the dataset where the target is  an outlier
print(f"Shape before removal of target outlier: {dataset_df.shape}")
mask = outlier_remover.fit_transform(dataset_df[[TARGET]])[TARGET].notna()
dataset_df = dataset_df.loc[mask, :]
print(f"Shape after removal of target outlier: {dataset_df.shape}")

## Create Sample object to be used by our model

In [ ]:
# Define a Sample object to be used by our model
sample_full = Sample(
    observations=dataset_df,
    target_name=TARGET)

* Plot of the empirical cumulative distribution function (ECDF) of the target 

In [ ]:
plot_ecdf(sample_full.target, iqr_multiple=1., iqr_multiple_far=None ,color_outlier='black', markersize=5)

# Boruta feature selection

## imputation and encoding
We first define a Pipeline that will be used before running Boruta

In [ ]:
def make_preprocessor(sample: Sample, onehot: bool = True, missing: bool = True) -> ColumnTransformerDF:
    tx = []
    # define how features should be preprocessed
    tx.append(("impute", 
               SimpleImputerDF(strategy="median"), 
               sample.features_by_type(Sample.DTYPE_NUMERICAL).columns))

    if missing:
        tx.append(("missing", 
                   MissingIndicatorDF(error_on_new=False), 
                   sample.features_by_type(Sample.DTYPE_NUMERICAL).columns))
        
    if onehot:
        tx.append(("onehot", 
                   OneHotEncoderDF(sparse=False, handle_unknown="ignore"),
                   sample.features_by_type([Sample.DTYPE_CATEGORICAL]).columns))     

    return ColumnTransformerDF(transformers=tx)

def make_outlier_transformer(sample: Sample, iqr_threshold) -> ColumnTransformerDF:    
    outlier_transformers = [
        ('outlier', TukeyOutlierRemoverDF(iqr_threshold=2), sample_full.features_by_type(Sample.DTYPE_NUMERICAL).columns),
        ('rest', FunctionTransformerDF(validate=False), sample_full.features_by_type(Sample.DTYPE_OBJECT).columns)
    ]
    outlier_step = ColumnTransformerDF(transformers=outlier_transformers)
    return outlier_step

In [ ]:
preprocessor = make_preprocessor(sample=sample_full, missing=True, onehot=True)
outlier_step = make_outlier_transformer(sample=sample_full, iqr_threshold=IQR_THRESHOLD)

## Feature selection (Boruta)

In [ ]:
boruta_selector = PipelineDF(
    steps = [
        ('outlier_removal', outlier_step),
        ('preprocess', preprocessor),
        ('boruta', BorutaDF(estimator=RandomForestRegressor(max_depth=5,min_samples_leaf=8,
                                                            random_state=42,n_jobs=-3),
                             n_estimators='auto', verbose=2, max_iter=100, random_state=42))])

In [ ]:
boruta_selector.fit(sample_full.features, sample_full.target)

In [ ]:
selected = sorted(list(set(boruta_selector.columns_original)))
print("List of the features selected by Boruta:\n")
pprint.pprint(list(selected))
if selected is None:
    raise Error("You need to specify a backup set of variables")

We then create a new sample object containing only the features selected by Boruta. 
- Before Boruta: 34 numercial features
- After Boruta: 18 features

In [ ]:
sample = sample_full.select_features(selected)

## Data Visualization of numerical features and categorical features

In [ ]:
plot_ecdf_df(sample.features)

# Model training


## Grid search and cross validator

In [ ]:
# define the circular cross validator with 30 folds
circular_cv = CircularCrossValidator(test_ratio=1/3, num_splits=3)

In [ ]:
def get_model_grids(sample: Sample) -> List[ModelGrid]:
    grids = [ModelGrid(
         model=Model(estimator=RandomForestRegressor(random_state=42),
                     preprocessing=make_preprocessor(sample=sample, missing=False)),
         estimator_parameters = {"n_estimators": [1000],
                                 "max_depth": [4,5,6],
                                 "min_samples_leaf": [4],
                                 "criterion": ["mse"], #["mae","mse"],
                                 "max_features": [0.4, 0.7, 1.0]})]
    return grids
grids = get_model_grids(sample)

In [ ]:
ranker = ModelRanker(grids=grids, cv=circular_cv)
ranking = ranker.run(sample, n_jobs=-3)
print(summary_report(ranking))

In [ ]:
top_model = ranking[0]
print(top_model.scoring['test_score'])
print(top_model.parameters)

## Model inspection

In [ ]:
predictor = PredictorCV(model=top_model.model, cv=circular_cv, sample=sample)
inspector = ModelInspector(predictor)
predictions = predictor.predictions_for_all_samples()

In [ ]:
plt.scatter(sample.target.sort_index(), 
            predictions.groupby(predictions.index)['prediction'].mean().sort_index(), alpha=.3)
plt.xlabel('actual')
plt.ylabel('predicted');

In [ ]:
predictions['error'] = (predictions['target'] - predictions['prediction'])
plt.scatter(x=predictions.index.values, y=predictions['error'].values, alpha=.2)
plt.xlabel("House index (ordered by sold date)")
plt.ylabel("Prediction error")
plt.axhline(0);

In [ ]:
M = inspector.shap_matrix()

In [ ]:
# number_features = predictor.sample.features.shape[1]
# _ = M.plot(subplots=True, figsize=(10, number_features*7))

In [ ]:
inspector.shap_matrix().head().T

In [ ]:
depm = inspector.feature_dependency_matrix()
plt.pcolormesh(depm)

In [ ]:
inspector.feature_importances().sort_values(ascending=False).to_frame()

## Shap clustering

In [ ]:
linkage_tree = inspector.cluster_dependent_features()

In [ ]:
number_features = predictor.sample.features.shape[1]
ax = plt.figure(figsize=(10, number_features*.5)).add_subplot(111)
style = FeatMapStyle(ax)
DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style).draw()

In [ ]:
ax = plt.figure(figsize=(10,number_features*0.5)).add_subplot(111)
style = LineStyle(ax)
DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style).draw()

# Shap clustering iterations

## Method the run an iteration

In [ ]:
def shap_clustering_iteration(sample: Sample, black_list: List[str]) -> Tuple[Sample, ModelInspector]:
    """Removing black list features, and retrain the model. Return a new sample and a fitted model inspector."""
    # Create a new Sample by removing the features in the blacklist
    features = sample.feature_names
    if not set(black_list) <= features:
        log.warning(f"""The black list must be a subset of the set of features: the features 
        {set(black_list)-set(features)} are in black list but not in the features.""")

    white_list = sorted(list(set(features) - set(black_list)))
    log.info(f"New white list:\n {white_list}")
    new_sample = sample.select_features(white_list)
    
    # Create the preprocessing pipeline for the model run
    outlier_step = make_outlier_transformer(sample=new_sample, iqr_threshold=IQR_THRESHOLD)
    preprocessor = make_preprocessor(sample=new_sample, missing=True, onehot=True)
    pipeline = PipelineDF(steps = [('outlier_removal', outlier_step), ('preprocess', preprocessor)])
    
    # Run the pipeline
    grids = get_model_grids(new_sample)
    ranker = ModelRanker(grids=grids, cv=circular_cv)
    ranking = ranker.run(new_sample, n_jobs=-3)
    top_model = ranking[0]
    
    # Report the model result
    print(summary_report(ranking))
    print(f"top_model score: {top_model.scoring['test_score']}")
    print(f"top_model parameters: {top_model.parameters}")
    predictor = PredictorCV(model=top_model.model, cv=circular_cv, sample=new_sample)
    inspector = ModelInspector(predictor)
    
    return new_sample, inspector

In [ ]:
def plot_shap_dendogram(inspector: ModelInspector) -> None:
    """Plot dendogram of the shape clustering."""
    M = inspector.shap_matrix()
    number_features = inspector.predictor.sample.features.shape[1]
    linkage_tree = inspector.cluster_dependent_features()
    ax = plt.figure(figsize=(10,number_features*.5)).add_subplot(111)
    style = FeatMapStyle(ax)
    DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style).draw();

## Shap culstering iteration 1

In [ ]:
black_list1 = ['WoodDeckSF', 
               #'BsmtFinSF1', 
               "OpenPorchSF",
               "MSSubClass",
               "KitchenAbvGr",
               'BsmtUnfSF', 
               #'MasVnrArea',
               'LotFrontage']

In [ ]:
sample1, inspector1 = shap_clustering_iteration(sample, black_list1)

In [ ]:
plot_shap_dendogram(inspector1)

# Shap clustering iteration 2

In [ ]:
black_list2 = [
    "YearRemodAdd",
    "OverallCond",
    "2ndFlrSF",
    #"MSSubClass",
    #"KitchenAbvGr"
]

In [ ]:
sample2, inspector2 = shap_clustering_iteration(sample1, black_list2)

In [ ]:
plot_shap_dendogram(inspector2)

# Simulation

In [ ]:
predictor = inspector2.predictor
sim = UnivariateSimulation(predictor=predictor)

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

dd = widgets.Dropdown(
    options=predictor.sample.features.columns,
    description='Feature:',
    disabled=False,
    layout={"width":"550px"}
)

btn = widgets.Button(description='Simulate')

def plot_simulation(feature:str):
    yield_change = sim.simulate_yield_change(
            parameterized_feature=feature,
            parameter_values=UnivariateSimulation.observed_feature_values(
                feature_name=feature,
                sample=predictor.sample,
                min_relative_frequency=0.03,
                limit_observations=100
            ),
    )
    
    yield_change_aggr = UnivariateSimulation.aggregate_simulated_yield_change(
                    results_per_split=yield_change, percentiles=[10, 50, 90])
    
    XLABEL_TITLE = f"{feature}"
    YLABEL_TITLE = f"Predicted mean yield uplift ({TARGET})"
    COLOR1 = 'red'
    COLOR2 = 'silver'
    
    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10,10), sharex=True)
    
    # plot lines of prediction
    ax1.set_xlabel(XLABEL_TITLE, color='black', labelpad=10, fontsize=12)
    ax1.set_ylabel(YLABEL_TITLE, color='black', fontsize=12)
    line1, = ax1.plot(yield_change_aggr.index, yield_change_aggr.iloc[:,0], color=COLOR2, linewidth=1)
    line2, = ax1.plot(yield_change_aggr.index, yield_change_aggr.iloc[:,1], color=COLOR1)
    line3, = ax1.plot(yield_change_aggr.index, yield_change_aggr.iloc[:,2], color=COLOR2, linewidth=1)
    ax1.axhline(y=0, color='black', linewidth=.5)
    ax1.tick_params(axis='x', labelcolor='black')
    for pos in ['top', 'right', 'bottom']:
        ax1.spines[pos].set_visible(False)
    ax1.tick_params(axis='x', labelbottom=True, bottom=False)
    ax1.legend((line3, line2, line1), ('90th percentile', 'Median', '10th percentile'), frameon=False)
    
    # plot the histogram
    x = sample.features[feature].dropna()
    hist_range = (min(yield_change_aggr.index), max(yield_change_aggr.index))
    n, bins, patches = ax2.hist(x, edgecolor='white', color=COLOR2, range=hist_range)
    bins1 = pd.Series(bins).rolling(window=2).mean().shift(-1).dropna()
    ax2.invert_yaxis()
    ax2.tick_params(axis='y', labelcolor='black')
    max_y = max(n)
    y_offset = max_y * 0.05
    for (x,y) in zip(bins1, n):
        if y>0:
            ax2.text(x, y + y_offset, str(int(y)), color='black', horizontalalignment='center')
    ax2.get_yaxis().set_visible(False)
    ax2.get_xaxis().set_visible(False)
    for pos in ['top', 'right', 'left', 'bottom']:
        ax2.spines[pos].set_visible(False)
    plt.subplots_adjust(hspace=.2)
    plt.show()

def on_click(btn):
    clear_output()
    display(widgets.HBox([dd, btn]))
    plot_simulation(feature=dd.value)
    
btn.on_click(on_click)    
display(widgets.HBox([dd, btn]))